# ElemNet training

# 1. SSUB

In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="ssub.json",url="https://figshare.com/ndownloader/files/40084921")


Loading the zipfile...
Loading completed.


In [2]:
import pandas as pd
df = pd.DataFrame(dat)

In [3]:
df

,id,formula_energy,formula
0,465,-2.841796e+00,CaO4W
1,1197,1.766051e-09,Ho
2,1759,-3.555857e-01,PbTe
3,1454,-9.540103e-01,MoS2
4,218,-6.564038e-02,BeH2
...,...,...,...
1722,1482,-1.892842e+00,NZr
1723,773,-2.404511e+00,CrLiO2
1724,251,-4.946426e-01,BiNa3
1725,328,-7.213532e-01,Br4Ge


In [4]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/ssub_formula_energy.json.zip"
js_tag = "ssub_formula_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [5]:
id_data.keys()

dict_keys(['train', 'test'])

In [6]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [7]:
print(len(train_ids),len(test_ids),len(train_ids)+len(test_ids))

1381 345 1726


In [8]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [9]:
len(train_df),  len(test_df)

(1381, 345)

In [10]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['formula_energy'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['formula_energy'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')


CPU times: user 209 ms, sys: 9 ms, total: 218 ms
Wall time: 216 ms


In [11]:
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [12]:
X_train.shape,X_test.shape

((1104, 103), (345, 103))

In [13]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-22 19:54:12.416289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 19:54:13.230240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-22 19:54:13.230329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-22 19:54:13.230339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [14]:
#Network from "Cross-property deep transfer learning framework for enhanced predictive analytics on small materials data"
#Link to paper: https://www.nature.com/articles/s41467-021-26921-5
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=False)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=False)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=False)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=False)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


2023-05-22 19:54:14.106933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 19:54:15.565745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14611 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-05-22 19:54:15.567063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14611 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


In [15]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000


2023-05-22 19:54:18.573038: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2f10036030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-22 19:54:18.573107: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-22 19:54:18.573132: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-22 19:54:18.587308: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-22 19:54:19.009193: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 - 7s - loss: 0.8622 - mean_absolute_error: 0.8622 - val_loss: 0.5779 - val_mean_absolute_error: 0.5779 - 7s/epoch - 196ms/step
Epoch 2/3000
35/35 - 0s - loss: 0.4997 - mean_absolute_error: 0.4997 - val_loss: 0.3375 - val_mean_absolute_error: 0.3375 - 436ms/epoch - 12ms/step
Epoch 3/3000
35/35 - 0s - loss: 0.3609 - mean_absolute_error: 0.3609 - val_loss: 0.3490 - val_mean_absolute_error: 0.3490 - 414ms/epoch - 12ms/step
Epoch 4/3000
35/35 - 0s - loss: 0.3700 - mean_absolute_error: 0.3700 - val_loss: 0.4310 - val_mean_absolute_error: 0.4310 - 427ms/epoch - 12ms/step
Epoch 5/3000
35/35 - 0s - loss: 0.3078 - mean_absolute_error: 0.3078 - val_loss: 0.2022 - val_mean_absolute_error: 0.2022 - 405ms/epoch - 12ms/step
Epoch 6/3000
35/35 - 0s - loss: 0.2717 - mean_absolute_error: 0.2717 - val_loss: 0.1690 - val_mean_absolute_error: 0.1690 - 432ms/epoch - 12ms/step
Epoch 7/3000
35/35 - 0s - loss: 0.1862 - mean_absolute_error: 0.1862 - val_loss: 0.1448 - val_mean_absolute_error: 0.1448 - 414

In [16]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 3ms/step
0.19525537929785258


In [17]:
# pred=model.predict(X_test)
# print(mean_absolute_error(y_test,pred))
# 0.17889024475861343

In [18]:
import os
f=open('AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred[0]):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv.zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv (deflated 4%)


0

# 2. SuperCon

In [19]:
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="supercon_chem.json",url="https://figshare.com/ndownloader/files/40719260")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)

Loading the zipfile...
Loading completed.


In [20]:
df

,id,Tc,formula
0,1524,0.00,Tb1Th3.4Ru0.6B4
1,14309,0.00,B2Th1
2,6607,1.09,Ni2.5Cu1.5Zr8
3,4010,24.60,Pr0.91La1Ce0.09Cu1O4
4,14511,55.00,Pb0.5Ca1Sr2Y0.5Cu2O6.97
...,...,...,...
16409,442,0.00,As1Pd7
16410,14942,0.00,La1.5Gd1.5Ca1Ba2Sr1Cu6.3In0.7O16.62
16411,3978,0.00,N0.98Pr1
16412,1078,32.00,Sm0.1La1.7Sr0.2Cu1O4


In [21]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/supercon_chem_Tc.json.zip"
js_tag = "supercon_chem_Tc.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [22]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['Tc'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['Tc'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [23]:
X_train.shape,X_test.shape

((10504, 103), (3283, 103))

In [24]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Network from "Cross-property deep transfer learning framework for enhanced predictive analytics on small materials data"
#Link to paper: https://www.nature.com/articles/s41467-021-26921-5
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=False)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=False)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=False)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=False)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [25]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
329/329 - 8s - loss: 12.5190 - mean_absolute_error: 12.5190 - val_loss: 15.3994 - val_mean_absolute_error: 15.3994 - 8s/epoch - 24ms/step
Epoch 2/3000
329/329 - 4s - loss: 10.3045 - mean_absolute_error: 10.3045 - val_loss: 9.9070 - val_mean_absolute_error: 9.9070 - 4s/epoch - 12ms/step
Epoch 3/3000
329/329 - 4s - loss: 9.4839 - mean_absolute_error: 9.4839 - val_loss: 9.1637 - val_mean_absolute_error: 9.1637 - 4s/epoch - 12ms/step
Epoch 4/3000
329/329 - 4s - loss: 9.2584 - mean_absolute_error: 9.2584 - val_loss: 10.9154 - val_mean_absolute_error: 10.9154 - 4s/epoch - 12ms/step
Epoch 5/3000
329/329 - 4s - loss: 9.2481 - mean_absolute_error: 9.2481 - val_loss: 8.5396 - val_mean_absolute_error: 8.5396 - 4s/epoch - 12ms/step
Epoch 6/3000
329/329 - 4s - loss: 8.7479 - mean_absolute_error: 8.7479 - val_loss: 9.1120 - val_mean_absolute_error: 9.1120 - 4s/epoch - 12ms/step
Epoch 7/3000
329/329 - 4s - loss: 8.5804 - mean_absolute_error: 8.5804 - val_loss: 8.2558 - val_mean_absolute_

In [26]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

103/103 [==============================] - 0s 2ms/step
5.11397283087764


In [27]:
import os
f=open('AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv.zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv (deflated 54%)


0

# 3. Magnetic2DChem

In [28]:
import json,zipfile
import numpy as np
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="mag2d_chem.json",url="https://figshare.com/ndownloader/files/40720004")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)
path = "../../benchmarks/AI/SinglePropertyPrediction/mag2d_chem_magnetic_moment.json.zip"
js_tag = "mag2d_chem_magnetic_moment.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

Loading the zipfile...
Loading completed.


In [29]:
df

,id,magnetic_moment,formula
0,49,5.0780,V1Cr1Ge2Te6
1,4,3.1560,Cr1Co1Si2Te6
2,212,3.0161,Ta1Cr1Sn1Ge1Te6
3,153,2.7459,Cr1Sn1Te6P1Au1
4,46,2.2800,Ti1Cr1Si2Te6
...,...,...,...
221,57,2.9670,Y1Cr1Si1Te6P1
222,97,5.6479,Cr1Ge2Te6W1
223,60,2.6981,Ta1Cr1In1Te6Pb1
224,152,5.0129,Cr1Sn1Te6Mo1P1


In [30]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['magnetic_moment'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['magnetic_moment'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [31]:
X_train.shape,X_test.shape

((144, 103), (46, 103))

In [32]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Network from "Cross-property deep transfer learning framework for enhanced predictive analytics on small materials data"
#Link to paper: https://www.nature.com/articles/s41467-021-26921-5
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = Activation('relu')(layer_1)

layer_2 = Dense(1024)(layer_1)
layer_2 = Activation('relu')(layer_2)

layer_3 = Dense(1024)(layer_2)
layer_3 = Activation('relu')(layer_3)

layer_4 = Dense(1024)(layer_3)
layer_4 = Activation('relu')(layer_4)
layer_4 = Dropout(0.2)(layer_4, training=False)

layer_5 = Dense(512)(layer_4)
layer_5 = Activation('relu')(layer_5)

layer_6 = Dense(512)(layer_5)
layer_6 = Activation('relu')(layer_6)

layer_7 = Dense(512)(layer_6)
layer_7 = Activation('relu')(layer_7)
layer_7 = Dropout(0.1)(layer_7, training=False)

layer_8 = Dense(256)(layer_7)
layer_8 = Activation('relu')(layer_8)

layer_9 = Dense(256)(layer_8)
layer_9 = Activation('relu')(layer_9)

layer_10 = Dense(256)(layer_9)
layer_10 = Activation('relu')(layer_10)
layer_10 = Dropout(0.3)(layer_10, training=False)

layer_11 = Dense(128)(layer_10)
layer_11 = Activation('relu')(layer_11)

layer_12 = Dense(128)(layer_11)
layer_12 = Activation('relu')(layer_12)

layer_13 = Dense(128)(layer_12)
layer_13 = Activation('relu')(layer_13)
layer_13 = Dropout(0.2)(layer_13, training=False)

layer_14 = Dense(64)(layer_13)
layer_14 = Activation('relu')(layer_14)

layer_15 = Dense(64)(layer_14)
layer_15 = Activation('relu')(layer_15)

layer_16 = Dense(32)(layer_15)
layer_16 = Activation('relu')(layer_16)

out_layer = Dense(1)(layer_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [33]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
5/5 - 4s - loss: 3.2914 - mean_absolute_error: 3.2914 - val_loss: 1.9751 - val_mean_absolute_error: 1.9751 - 4s/epoch - 840ms/step
Epoch 2/3000
5/5 - 0s - loss: 1.3997 - mean_absolute_error: 1.3997 - val_loss: 0.9263 - val_mean_absolute_error: 0.9263 - 107ms/epoch - 21ms/step
Epoch 3/3000
5/5 - 0s - loss: 0.8969 - mean_absolute_error: 0.8969 - val_loss: 0.8569 - val_mean_absolute_error: 0.8569 - 118ms/epoch - 24ms/step
Epoch 4/3000
5/5 - 0s - loss: 0.9019 - mean_absolute_error: 0.9019 - val_loss: 0.7718 - val_mean_absolute_error: 0.7718 - 118ms/epoch - 24ms/step
Epoch 5/3000
5/5 - 0s - loss: 0.8842 - mean_absolute_error: 0.8842 - val_loss: 0.8025 - val_mean_absolute_error: 0.8025 - 95ms/epoch - 19ms/step
Epoch 6/3000
5/5 - 0s - loss: 0.9628 - mean_absolute_error: 0.9628 - val_loss: 0.9974 - val_mean_absolute_error: 0.9974 - 84ms/epoch - 17ms/step
Epoch 7/3000
5/5 - 0s - loss: 1.0290 - mean_absolute_error: 1.0290 - val_loss: 0.9094 - val_mean_absolute_error: 0.9094 - 101ms/

In [34]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

2/2 [==============================] - 0s 4ms/step
0.44767221206996743


In [35]:
import os
f=open('AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv.zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv (deflated 50%)


0